In [1]:
!pip install python-dotenv

  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)


ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\tools\\Anaconda3\\Lib\\site-packages\\dotenv'
Consider using the `--user` option or check the permissions.



In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns





In [2]:
columns_to_keep_games_stats = ["PlayerId", "CreatedAt", "UpdatedAt", "Map",  "Kills", "Deaths", "Assists", "Headshots", "FirstHalfScore", "SecondHalfScore",'QuadroKills', 'TripleKills', 'PentaKills',"Headshots", "Headshots%","K/rRatio", "K/dRatio", 'Mvps', "OvertimeScore", "Rounds", "Score", "Result"]

df = pd.read_pickle("./dataCompressed.pkl", "zip")[columns_to_keep_games_stats]


In [3]:
type_casting = {"PlayerId": str, "CreatedAt": "datetime64[ns]", "UpdatedAt":  "datetime64[ns]", "Kills": int, "Deaths": int, "Assists": int, "Headshots": int, "FirstHalfScore":int, "SecondHalfScore": int, "QuadroKills": int,  "TripleKills": int, "PentaKills": int, "Mvps": int, "OvertimeScore": int, "Rounds": int, "Result": int}
df = df.astype(type_casting, errors = "ignore")


In [4]:
df.dropna(how = "any", inplace=True)
df["Result"] = df["Result"].astype(int).astype(bool)

In [5]:
df.shape

(398294, 22)